In [4]:
import pandas as pd
import os
from konlpy.tag import Mecab
mecab = Mecab()
import re

In [5]:
os.getcwd()
%ls -al


total 40
drwxr-xr-x  4 sjham sjham 4096 Jan 27 23:24 ./
drwxr-xr-x 14 sjham sjham 4096 Jan 27 21:53 ../
drwxr-xr-x  2 sjham sjham 4096 Jan 27 23:19 data_set/
-rw-r--r--  1 sjham sjham  344 Jan 27 22:03 matcher.py
-rw-r--r--  1 sjham sjham  412 Jan 27 22:57 match_to_col.py
drwxr-xr-x  2 sjham sjham 4096 Jan 27 23:24 __pycache__/
-rw-r--r--  1 sjham sjham  319 Jan 27 23:14 reader.py
-rw-r--r--  1 sjham sjham  250 Jan 27 23:00 save.py
-rw-r--r--  1 sjham sjham  392 Jan 27 21:59 tokenizer.py
-rw-r--r--  1 sjham sjham  364 Jan 27 22:56 vecterizer.py


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
data = pd.read_excel('broadcast.xlsx') 
# data = pd.read_csv('manual.csv') 
raw_data = pd.read_csv('kbs_2016,3,2_to_2016,4,30.csv')

In [ ]:
# chunksize = 500
# chunks = []
# for chunk in pd.read_csv('pizza.csv', chunksize=chunksize):
#     # Do stuff...
#     chunks.append(chunk)

# df = pd.concat(chunks, axis=0)


In [ ]:
data.head()
raw_data.head()


In [ ]:
# raw_data['date'] = pd.to_datetime(raw_data['date'])

In [ ]:
# raw_data['date'] = raw_data['date'].dt.date

In [ ]:
# data['date'] = '2016.' + data['date'].astype(str)

In [ ]:
# data['date'] = data['date'].str.replace('/', '.').str.replace('-', '.')

In [ ]:
# data['date']= pd.to_datetime(data['date'])

In [ ]:
# data.set_index('date')
# raw_data.set_index('date')

In [ ]:
# data['title_refined'] = data.제목.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")
# raw_data['title_refined']= raw_data.title.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")

In [ ]:
# data['title_refined'] = data['title_refined'].str.strip()
# raw_data['title_refined'] = raw_data['title_refined'].str.strip()

In [ ]:
# data.isnull().sum()
# raw_data.isnull().sum()

In [ ]:
# data = data.dropna(subset=['title_refined'],axis=0, how='any')
# data = data.dropna(subset=['제목'],axis=0, how='any')

In [ ]:
# data.drop('title',axis=1, inplace=True)

In [ ]:
# def ngrams(string, n=3):
# #     string = re.sub(r'[,-./]|\sBD',r'', string)
#     ngrams = zip(*[string[i:] for i in range(n)])
#     return [''.join(ngram) for ngram in ngrams]

In [ ]:
# def tagging(x):
#     return mecab.nouns(x)
#     return x+'1'

In [ ]:
# def tagging(x):
#     return mecab.morphs(x)

In [1]:
%%writefile '~/project/nlp/preprocessing/preprocess.py'

import pandas as pd
import numpy as np
# import os
from konlpy.tag import Mecab
import re
from difflib import SequenceMatcher
mecab = Mecab()


#reading file

def read_file(file1, file2):
    if 'xlsx' in str(file1): 
        data1 = pd.read_excel(str(file1))
    else:
        data1 = pd.read_csv(str(file1))
    if 'xlsx' in str(file2):
        data2 = pd.read_excel(str(file2))
    else:
        data2 = pd.read_csv(str(file2))
    return data1, data2


#cleaning data

def cleaning_data(data1, data2):
    data1['title_refined'] = data1.제목.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")
    data2['title_refined'] = data2.title.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")
    data1['title_refined'] = data1['title_refined'].str.strip()
    data2['title_refined'] = data2['title_refined'].str.strip()
    return (data1, data2)


#tokenizing

def ngrams(string, num_ngram): 
#     string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(num_ngram)])
    return [''.join(ngram) for ngram in ngrams]
# print(ngrams(string))

def tagging_morphs(string):
    return mecab.morphs(string)

def tagging_nouns(string):
    return mecab.nouns(string)



# finding matching data

def intersect(data1, data2):
    inter = []
    for i in data1:
        for j in data2:
            inter.append(len(set(i).intersection(set(j))))
    return inter

def similar(data1, data2):
    sim=[]
#     threshold = 0.4
    for i in data1:
        for j in data2:
            t.append(SequenceMatcher(None, i, j).ratio())
    return sim


#vecterize and send to columns

def vecterize_to_cols(data1_col_tagged, data2_col_tagged):
    vect = np.array(similar(data1_col_tagged, data2_col_tagged)).reshape(len(data1.shape),len(data2.shape))
    arg_count = np.argmax(vect, axis=1)
    max_count = np.max(vect, axis=1)
    data1['arg_count']=pd.Series(arg_count)
    data1['max_count']=pd.Series(max_count)
    return


#matched data to columns

def match_to_cols(data1, data2):
    matching_title= []
    matching_aT = []
    t = int(data2.columns.get_loc('title'))
    aT = int(data2.columns.get_loc('articleText')) 
    for i in data1.arg_count:
        matching_title.append(data2.iloc[i,t])
        matching_aT.append(data2.iloc[i,aT])
    data1['matching_title'] = matching_title
    data1['matching_aT'] = matching_aT
    return


#save to file

def save_to_file(data1, file_name):
    pd.to_pickle(data1, file_name+'.pkl')
    data['verify'] = 0
    file_name = data[['제목', 'matching_title', 'verify','max_count', 'matching_aT']]
    file_name.to_csv(file_name+'.csv')




Overwriting /home/sjham/project/nlp/preprocessing/preprocess.py


In [13]:
%%writefile '~/project/nlp/preprocessing/reader.py'
import pandas as pd

def read_file(file1, file2):
    if 'xlsx' in str(file1): 
        data1 = pd.read_excel(str(file1))
    else:
        data1 = pd.read_csv(str(file1))
    if 'xlsx' in str(file2):
        data2 = pd.read_excel(str(file2))
    else:
        data2 = pd.read_csv(str(file2))
    return (data1, data2)  

Overwriting /home/sjham/project/nlp/preprocessing/reader.py


In [14]:
%%writefile '~/project/nlp/preprocessing/cleaner.py'
import re
def cleaning_data(data1, data2):
    data1['title_refined'] = data1.제목.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")
    data2['title_refined'] = data2.title.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")
    data1['title_refined'] = data1['title_refined'].str.strip()
    data2['title_refined'] = data2['title_refined'].str.strip()
    return 



# data = data.dropna(subset=['제목'],axis=0, how='any')


Overwriting /home/sjham/project/nlp/preprocessing/cleaner.py


In [17]:
%%writefile '~/project/nlp/preprocessing/tokenizer.py'
import pandas as pd
# import os
from konlpy.tag import Mecab
# import re

mecab = Mecab()

def ngrams(string, num_ngram): 
#     string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(num_ngram)])
    return [''.join(ngram) for ngram in ngrams]
# print(ngrams(string))

def tagging_morphs(string):
    return mecab.morphs(string)

def tagging_nouns(string):
    return mecab.nouns(string)






Overwriting /home/sjham/project/nlp/preprocessing/tokenizer.py


In [ ]:
%%writefile '~/project/nlp/preprocessing/matcher.py'

from difflib import SequenceMatcher

def intersect(data1, data2):
    inter = []
    for i in data1:
        for j in data2:
            inter.append(len(set(i).intersection(set(j))))
    return inter

def similar(data1, data2):
    sim=[]
#     threshold = 0.4
    for i in data1:
        for j in data2:
            t.append(SequenceMatcher(None, i, j).ratio())
    return sim


In [ ]:
%%writefile '~/project/nlp/preprocessing/vecterizer.py'

import numpy as np

def vecterize_to_cols(data1.clo_tagged, data2.col_tagged):
    vect = np.array(similar(data1.clo_tagged, data2.col_tagged)).reshape(len(data1.shape),len(data2.shape))
    arg_count = np.argmax(vect, axis=1)
    max_count = np.max(vect, axis=1)
    data1['arg_count']=pd.Series(arg_count)
    data1['max_count']=pd.Series(max_count)
    return


In [ ]:
%%writefile '~/project/nlp/preprocessing/match_to_col.py'

import pandas as pd

def match_to_cols(data1, data2):
    matching_title= []
    matching_aT = []
    t = int(data2.columns.get_loc('title'))
    aT = int(data2.columns.get_loc('articleText')) 
    for i in data1.arg_count:
        matching_title.append(data2.iloc[i,t])
        matching_aT.append(data2.iloc[i,aT])
    data1['matching_title'] = matching_title
    data1['matching_aT'] = matching_aT
    return


In [ ]:
%%writefile '~/project/nlp/preprocessing/save.py'

import pandas as pd

def save_to_file(data1, file_name):
    pd.to_pickle(data1, file_name+'.pkl')
    data['verify'] = 0
    file_name = data[['제목', 'matching_title', 'verify','max_count', 'matching_aT']]
    file_name.to_csv(file_name+'.csv')


In [ ]:
###########unrefined_morphs, refined_morphs, refined_nouns, ngrams  

import numpy as np

##refined
data = data.dropna(subset=['제목'],axis=0, how='any')

data['title_refined'] = data.제목.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")
raw_data['title_refined']= raw_data.title.str.replace(r'[/W\”\“\‘\,\’\"\'\.\∙\…\[\]\(\)]+', " ")

data['title_refined'] = data['title_refined'].str.strip()
raw_data['title_refined'] = raw_data['title_refined'].str.strip()



def ngrams(string, n=3): 
#     string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]
# print(ngrams(string))

def tagging_morphs(x):
    return mecab.morphs(x)

def tagging_nouns(x):
    return mecab.nouns(x)

data['title_tagged'] = data['title_refined'].apply(ngrams)
raw_data['title_tagged'] = raw_data['title_refined'].apply(ngrams)

# data['title_tagged'] = data['제목'].apply(tagging_morphs)
# raw_data['title_tagged'] = raw_data['title'].apply(tagging_morphs)

# data['title_tagged'] = data['title_refined'].apply(tagging_nouns)
# raw_data['title_tagged'] = raw_data['title_refined'].apply(tagging_nouns)



data = data[0:198]

def intersect(a, b):
    inter = []
    for i in a:
        for j in b:
            inter.append(len(set(i).intersection(set(j))))
    return inter

# intersect(data.title_tagged, raw_data.title_tagged)

# y= []
# for i in data.title_tagged:
#     for j in raw_data.title_tagged:
#         y.append(len(set(i).intersection(set(j))))

from difflib import SequenceMatcher
def similar(a, b):
    t=[]
#     threshold = 0.4
    for i in a:
        for j in b:
            t.append(SequenceMatcher(None, i, j).ratio())
            
#             > threshold:
#             inter.append(len(set(i).intersection(set(j))))
#     if SequenceMatcher(None, a, b).ratio() > threshold:
#         t.append(SequenceMatcher(None, a, b).ratio())
        #     print(t)
    return t
        
la = data.shape[0]
lra = raw_data.shape[0]

# n = np.array(intersect(data.title_tagged, raw_data.title_tagged)).reshape(la,lra)
n = np.array(similar(data.title_tagged, raw_data.title_tagged)).reshape(la,lra)

arg_count = np.argmax(n, axis=1)
max_count = np.max(n, axis=1)

data['arg_count']=pd.Series(arg_count)
data['max_count']=pd.Series(max_count)

data['articleText'] = 0

matching_aT = []
matching_title= []
for i in data.arg_count:
    matching_title.append(raw_data.iloc[i,0])
    matching_aT.append(raw_data.iloc[i,5])

data['matching_title'] = matching_title
data['matching_aT'] = matching_aT

# pd.to_pickle(data, 'refined_nouns.pkl')
# pd.to_pickle(data, 'unrefined_morphs.pkl')
# pd.to_pickle(data, 'refined_morphs.pkl')
pd.to_pickle(data, 'refined_3grams_similar.pkl')
# pd.to_pickle(data, 'refined_4grams.pkl')
# pd.to_pickle(data, 'refined_4grams_similar.pkl')
# pd.to_pickle(data, 'refined_morphs_similar.pkl')
# pd.to_pickle(data, 'refined_nouns_similar.pkl')


# refined_nouns = data[['제목', 'matching_title', 'max_count', 'matching_aT']]
# refined_nouns.to_csv('refined_nouns.csv')

# unrefined_morphs = data[['제목', 'matching_title', 'max_count', 'matching_aT']]
# unrefined_morphs.to_csv('unrefined_morphs.csv')

# refined_morphs = data[['제목', 'matching_title', 'max_count', 'matching_aT']]
# refined_morphs.to_csv('refined_morphs.csv')

# data['verify'] = 0
# refined_4grams = data[['제목', 'matching_title', 'verify','max_count', 'matching_aT']]
# refined_4grams.to_csv('refined_4grams.csv')

# data['verify'] = 0
# refined_4grams_similar = data[['제목', 'matching_title', 'verify','max_count', 'matching_aT']]
# refined_4grams_similar.to_csv('refined_4grams_similar.csv')

# data['verify'] = 0
# refined_morphs_similar = data[['제목', 'matching_title', 'verify','max_count', 'matching_aT']]
# refined_morphs_similar.to_csv('refined_morphs_similar.csv')

# data['verify'] = 0
# refined_nouns_similar = data[['제목', 'matching_title', 'verify','max_count', 'matching_aT']]
# refined_nouns_similar.to_csv('refined_nouns_similar.csv')

data['verify'] = 0
refined_3grams_similar = data[['제목', 'matching_title', 'verify','max_count', 'matching_aT']]
refined_3grams_similar.to_csv('refined_3grams_similar.csv')


In [ ]:
refined_nouns_similar

In [ ]:
refined_2grams.loc[refined_2grams.max_count <= 4, :].shape

In [ ]:
refined_4grams.loc[refined_4grams.max_count <= 4, :].shape

In [ ]:
refined_3grams.loc[refined_3grams.max_count <= 4, :].to_csv('refined_3grams_under4.csv')

In [ ]:
refined_3grams.loc[refined_3grams.max_count <= 4, :].shape

In [ ]:
refined_morphs.loc[refined_morphs.max_count <= 4, :].to_csv('refined_morphs_under4.csv')

In [ ]:
refined_morphs.loc[refined_morphs.max_count <= 4, :].shape

In [ ]:
refined_nouns.loc[refined_nouns.max_count <= 4, :].to_csv('refined_nouns_under4.csv')

In [ ]:
refined_nouns.loc[refined_nouns.max_count <= 4, :].shape

In [ ]:
unrefined_morphs.loc[unrefined_morphs.max_count <= 4, :].to_csv('unrefined_morphs_under4.csv')

In [ ]:
unrefined_morphs.loc[unrefined_morphs.max_count <= 4, :].shape

In [ ]:
unrefined_morphs.loc[unrefined_morphs.max_count <= 4, :]

In [ ]:
string = '치킨 게임으로 비화된 한일 초계기 갈등…제동장치 없나'
#       ['일본 인플루엔자 한국 영향 촉각… 백신 접종 당부'],
#       ['미 최장 셧다운 일단 수습…북·미 정상회담 준비 집중'],
#       ['자리 양보받으면 기분 나빠…58년 개띠 新중년'],
#       ['일요일 아침까지 추워…체감 영하 10도 안팎']]

    

In [ ]:
def ngrams(string, n=3): 
#     string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]
print(ngrams(string))

In [ ]:
def ngrams(string, n=3):
    for st in string:
        ngrams = zip(*[str(st)[i:] for i in range(n)])
        for ngram in ngrams:
            return ngram
#         return [''.join(ngram) for ngram in ngrams]
print(ngrams(string))

In [ ]:
# data['title_tagged'] = data['title_refined'].apply(tagging).copy()
data['title_tagged'] = data['제목'].apply(tagging)

In [ ]:
# data['title_set']= [set(item) for item in data['title_tagged']]

In [ ]:
# raw_data.drop('title', axis=1, inplace=True)

In [ ]:
# raw_data['title_tagged'] = raw_data['title_refined'].apply(tagging)
raw_data['title_tagged'] = raw_data['title'].apply(tagging)

In [ ]:
# raw_data['title_set']= [set(item) for item in raw_data['title_tagged']]

In [ ]:
# raw_data_tagged = raw_data.drop(['title_refined', 'title_set'], axis=1) 

In [ ]:
# raw_data_tagged.to_csv('raw_data_tagged.csv')

In [ ]:
# data_tagged = data.drop(['title_refined', 'title_set'], axis=1) 

In [ ]:
# data_tagged.to_csv('data_tagged.csv')

In [ ]:
# data.head()

In [ ]:
a = [['사랑', '암', '온정', '시작'], ['차량', '위반','암행', '단속','사랑','운행'],['암행', '시작', '행성']]
b = [['무제한', '위반', '온정'] ,['사랑', '암', '나를', '온정'],['토론','단속', '시작', '끝'], ['테러', '방지법'], ['등', '운행', '처리','사죄', '무제한', '암행','차량']]


In [ ]:
from difflib import SequenceMatcher
def similar(a, b):
    t=[]
#     threshold = 0.4
    for i in a:
        str(i)
        for j in b:
            str(j)
            t.append(SequenceMatcher(None, i, j).ratio())
print(similar(a, b))

In [ ]:
# df = pd.DataFrame({'manual':a})
# df2 =pd.DataFrame({'crawl':b}) 

In [ ]:
data.head()
raw_data.head()

In [ ]:
data = data[0:198]

In [ ]:
y= []
for i in data.title_tagged:
    for j in raw_data.title_tagged:
        y.append(len(set(i).intersection(set(j))))
len(y)
y

la = data.shape[0]
lra = raw_data.shape[0]

import numpy as np
n = np.array(y).reshape(la,lra)
n
arg_count = np.argmax(n, axis=1)
max_count = np.max(n, axis=1)
arg_count
max_count

In [ ]:
data['arg_count']=pd.Series(arg_count)
data['max_count']=pd.Series(max_count)

In [ ]:
data['articleText'] = 0

In [ ]:
# df= df.drop(['count','real_count'],axis=1)

In [ ]:
# df= df.drop(['text'],axis=1)

In [ ]:
raw_data.head()

In [ ]:
matching_data = []
for i in data.arg_count:
    matching_data.append(raw_data.iloc[i,0])
data['articleText'] = matching_data

In [ ]:
data

In [ ]:
data[['제목','max_count', 'articleText']]

In [ ]:
pd.to_pickle(data, 'with_real_noncharccter.pkl')

In [ ]:
pd.to_pickle(data, 'with_noncharccter.pkl')


In [ ]:
pd.to_pickle(data, 'data_tagged_aT.pkl')

In [ ]:
pickled_data = pd.read_pickle('./with_real_noncharccter.pkl')

In [ ]:
pickled_data.head()

In [ ]:
noncharccter_data_at_max = pickled_data[['제목','max_count', 'articleText']]

In [ ]:
noncharccter_data_at_max.to_csv('noncharccter_data_at_max')

In [ ]:
noncharccter_data_at_max.loc[noncharccter_data_at_max.max_count ==4, :]


In [ ]:
tagged_data_at_max.to_csv('tagged_data_at_max')

In [ ]:
tagged_data_at_max.loc[tagged_data_at_max.max_count ==4, :]

In [ ]:
nontagged_data_at_max.to_csv('nontagged_data_at_max')

In [ ]:
nontagged_data_at_max.loc[nontagged_data_at_max.max_count <= 3, :].shape

In [ ]:
data

In [ ]:
pickled_data['articleText']

In [ ]:
data['제목_splited'] = [set(str(i).split()) for i in data.제목_refined]
raw_data['title_splited'] = [set(str(i).split()) for i in raw_data.title_refined]


In [ ]:
matched = set1.intersection(set2)

import numpy as np
import pandas as pd

a = [['10', '1.2', '4.2'], ['15', '70', '0.03'], ['8', '5', '0']]
df = pd.DataFrame(a, columns=['one', 'two', 'three'])

df['compare'] = np.where((data['date'] == raw_data['date']) & (data['date'] <= df['three'])
                     , df['one'], np.nan)

In [ ]:
a = '치킨 게임으로 인플루엔자 비화된 한일 초계기 갈등…제동장치 , 백신 접종 당부'
b = '일본 인플루엔자 한국 영향 촉각… 백신 접종 당부'
#       ['미 최장 셧다운 일단 수습…북·미 정상회담 준비 집중'],
#       ['자리 양보받으면 기분 나빠…58년 개띠 新중년'],
#       ['일요일 아침까지 추워…체감 영하 10도 안팎']]


In [ ]:
from difflib import SequenceMatcher
def similar(a, b):
    t=[]
    threshold = 0.4
    if SequenceMatcher(None, a, b).ratio() > threshold:
        t.append(SequenceMatcher(None, a, b).ratio())
#     print(t)
    return t
print(similar(a, b))        
#     print(s)
#     return s

In [ ]:
from difflib import SequenceMatcher
SequenceMatcher(None, raw_data.title_tagged, data.title_tagged)


In [ ]:
for rd in raw_data.title_tagged:
    for d in data.title_tagged:
        similar(rd, d)

In [ ]:
# data['제목_splited'] = [set(str(i).split()) for i in data.제목_refined]
# raw_data['title_splited'] = [set(str(i).split()) for i in raw_data.title_refined]

data['제목_splited'] = [str(i).split() for i in data.제목_refined]
raw_data['title_splited'] = [str(i).split() for i in raw_data.title_refined]


In [ ]:
data_100 = data['제목_splited'][:100] 
raw_data_100 = raw_data['title_splited'][:100]

In [ ]:
a = [['시간', '유세', '돌입', '군소정당', '총력전'], ['총선', '이틀', '앞', '5대', '관전', '포인트는?'], 
     ['반드시', '투표', '비', '예보', '변수'], ['與', '과반', '확보', '절실', '투표로', '야당', '심판','컷오프', '내일 발표'],
     ['윤상현 막말', '파문…','컷오프', '내일 발표']]


# raw_data['title_splited'] = [','.join(str(i).split()) for i in raw_data.title_refined]
# raw_data['title_fil'] = [set(str(i).split()) for i in raw_data.title]

# [더민주, 여당, 독주, 저지, 수권, 야당, 지지], [국민의당, 미래, 위한, 싸움, 3번, 선택], 
#      [우리가, 대안, 정당, 존재감, 부각, 총력], [내일, 선택의, 날, 투, 개표, 준비, 완료],
#  [투표용지, 손동작, 안, 돼, 인증샷, 주의점], [오후, 6시, 출구조사, 당선, 예상자, 발표]]
b = [['시간', '유세', '돌입', '군소정당', '총력전'],['與', '과반', '확보', '절실'], ['심판','컷오프', '내일 발표'],
     ['yyyyyyyy','ttttt','wswwewe'],['dwddwdfwfw','gfhfghhth','hytyyhtyhytyh']]

    
    
#     [더민주, 여당, 독주, 저지, 수권, 야당, 지지], [국민의당, 미래, 위한, 싸움, 3번, 선택], 
#      [우리가, 대안, 정당, 존재감, 부각, 총력],


In [ ]:
data['날짜_in'] = data['날짜'].astype('str').str.replace('-', '')
# df.to_csv(r'c:\data\pandas.txt', header=None, index=None, sep=' ', mode='a')
# with open('dates_kbs.txt', 'w') as f:
#     f.write(data['날짜_in'].astype(int).values)

# data['날짜_in'].astype(int).values

In [ ]:
# import numpy as np
# nparray = data['날짜_in'].values
# np.savetxt('dates_kbs.txt', nparray, delimiter=",")
# nparray

In [ ]:
data.날짜_in.to_csv('dates_kbs.csv', header=None, index=None, sep=',')
# import numpy as np
# np.savetxt('dates_kbs.csv', data.날짜_in.values, fmt='%d')

# data.날짜_in.to_csv('date.csv')

In [ ]:
csv_file = '/home/sjham/project/scrapy/MyProject/data_set/dates_kbs.csv'
a = pd.read_csv(csv_file, header=None, index_col=None)
for i in a.values:
    int(i)


In [ ]:
# data['제목_splited'] = [','.join(str(i).split()) for i in data.제목_refined]
# data['제목_fil'] = [set(str(i).split()) for i in data.제목]

# .apply(split()))

# for n in data.제목_refined:
#     a = mecab.nouns(str(n))
    
# [len(set(a).intersection(b)) for a, b in zip(df.A, df.B)]    
    
#     pd.Series([mecab.nouns(n)])

In [ ]:
# for rd in raw_data['title_splited']:
#     str(rd)
#     for d in data['제목_splited']:
#         str(d)
#         print(similar(rd, d))
        

In [ ]:
filter =[]
for rd in data['제목_fil']:
    for d in raw_data['title_fil']:
        if len(d & rd) > 3:
            filter.append(d)
            
            
#             raw_data['filtered'] = d
            
        
        
#         raw_data.loc[len(d & rd) > 3, :]
#         if len(d & rd) > 3:
#             raw_data['title_splited']
            

In [ ]:



df1 = pd.DataFrame({'c1': [1, 4, 7], 'c2': [2, 5, 1], 'c3': [3, 1, 1]})
df2 = pd.DataFrame({'c4': [1, 4, 7], 'c2': [3, 5, 2], 'c3': [3, 7, 5]})
set(df1['c2']).intersection(set(df2['c2']))



In [ ]:
set(map(tuple, my_list))

In [ ]:
sorted(
    
    set(map(tuple, my_list)), reverse=True)
result
result = sorted(set(map(tuple, my_list)), reverse=True)
result


In [ ]:
set(raw_data['title_splited']).intersection(set(data['제목_splited']))

In [ ]:
# df_merged = data.merge(raw_data, how='inner', left_on='제목', right_on='title')


In [ ]:
# df_merged.head()